In [12]:
import pandas as pd
import openai

with open('token.txt') as f:
    api_key = f.readline()

openai.api_key = api_key

# list models
models = openai.Model.list()
models = pd.DataFrame(models.data)
models['created'] = pd.to_datetime(models['created'], unit='s')
models.sort_values('id', inplace=True)
models[models['id'].str.contains('gpt')]

,id,object,created,owned_by,permission,root,parent
4,gpt-3.5-turbo,model,2023-02-28 18:56:42,openai,"[{'id': 'modelperm-aRL68Vu37kh2Z0wi2tmPhGZT', ...",gpt-3.5-turbo,None
56,gpt-3.5-turbo-0301,model,2023-03-01 05:52:43,openai,"[{'id': 'modelperm-I4IcSJFYZl2fIK0DPSBkgK3d', ...",gpt-3.5-turbo-0301,None
13,gpt-3.5-turbo-0613,model,2023-06-12 16:30:34,openai,"[{'id': 'modelperm-t8jT1BkYxMNFFB57ndHNpyfP', ...",gpt-3.5-turbo-0613,None
19,gpt-3.5-turbo-16k,model,2023-05-10 22:35:02,openai-internal,"[{'id': 'modelperm-BPjHua9GyiXIbj7F0M6c4j0A', ...",gpt-3.5-turbo-16k,None
15,gpt-3.5-turbo-16k-0613,model,2023-05-30 19:17:27,openai,"[{'id': 'modelperm-hb67ObpoaFIKTieIMs9CqZPq', ...",gpt-3.5-turbo-16k-0613,None
1,gpt-4,model,2023-06-27 16:13:31,openai,"[{'id': 'modelperm-tsET7VSHRhiQHP5Qz6TU1FBb', ...",gpt-4,None
39,gpt-4-0314,model,2023-06-27 16:13:30,openai,"[{'id': 'modelperm-AK02E3SrXhuSjwJAWvcSvR3q', ...",gpt-4-0314,None
7,gpt-4-0613,model,2023-06-12 16:54:56,openai,"[{'id': 'modelperm-XgbDsLBjXy3aVd6xZvWJFmW2', ...",gpt-4-0613,None


In [14]:
models[models['id'].str.contains('embedding')]

,id,object,created,owned_by,permission,root,parent
30,text-embedding-ada-002,model,2022-12-16 19:01:39,openai-internal,"[{'id': 'modelperm-6xIVIGILGxg9gTVWY0SpzkFn', ...",text-embedding-ada-002,None


In [46]:
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=api_key, model_name='gpt-4-0613')

In [97]:
def get_cat4_test_q_prompt(n_questions=5, user_instruction=""):

    ordered_instructions = [
        f"Generate JSON with list of {n_questions} questions.",
        """Each questions is a dict with such keys:
        question_id (string), question (string), options (list of strings), correct_answer (string), explanation (string).""",
        "Each question should only contain one correct answer. Incorrect answers should be plausible but incorrect.",
        "Provide explanation for the correct answer.",
        """Format the output as in the example:

        {{
            "questions": [
                {{
                    "question_id": 1,
                    "question": "Which one of the following is not a bird?",
                    "options": ["🦅", "🦉", "🦚", "🐱"],
                    "correct_answer": "🐱",
                    "explanation": "birds have wings, 🐱 does not"
                }},
                {{
                    "question_id": 2,
                    "question": "Which animal is known to love eating bananas?",
                    "options": ["🐶", "🐱", "🦓", "🐒"],
                    "correct_answer": "🐒",
                    "explanation": "it is hard for other animals to grasp a banana, so they usually eat other food"
                }}
            ]
        }}

        """,
        """Only use the following context (if exists) to come up with the questions:
        {text}""",
    ]

    prompt_template = "Abide by instructions.\n"
    instructions_counter = 1
    if len(user_instruction):
        prompt_template += f"Instruction {instructions_counter}. {user_instruction}\n"
        instructions_counter += 1

    for instruction in ordered_instructions:
        prompt_template += f"Instruction {instructions_counter}. {instruction}\n"
        instructions_counter += 1


    prompt_template += "\nJSON:"

    main_prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

    refine_template = (
        "Your job is to refine a quiz with single-choice questions for children.\n"
        "We have provided an existing set of questions in JSON: {existing_answer}\n"
        "Here is some more context (if exists) that might help.\n"
        "------------\n"
        "{text}\n"
        "------------\n"
        "Given the new context, refine the original set of questions.\n"
        "If the context isn't useful, return the original set of questions.\n"
        "Only respond with a single JSON.\n"
        "Here is the refined quiz in JSON format based on the provided context:"
    )

    refined_prompt = PromptTemplate(
        input_variables=["existing_answer", "text"],
        template=refine_template,
    )

    return main_prompt, refined_prompt

# Example usage
main_prompt, refined_prompt = get_cat4_test_q_prompt(
    n_questions=5,
    user_instruction="Create a Verbal Reasoning question using Animal emojis for 11+ United Kingdom exam."
)
# print("Main Prompt:", main_prompt.template)
# print("Refined Prompt:", refined_prompt.template)

In [98]:
docs = [""]
docs = [Document(page_content=doc_text) for doc_text in docs]

In [99]:
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    return_intermediate_steps=True,
    question_prompt=main_prompt,
    refine_prompt=refined_prompt,
)
#
output = chain({"input_documents": docs}, return_only_outputs=True)

In [100]:
print(output['output_text'])

{
    "questions": [
        {
            "question_id": "1",
            "question": "Which of the following animals is not a pet?",
            "options": ["🐶", "🐱", "🐹", "🦁"],
            "correct_answer": "🦁",
            "explanation": "A lion (🦁) is a wild animal and not typically kept as a pet."
        },
        {
            "question_id": "2",
            "question": "Which one of these animals is known to live in water?",
            "options": ["🐮", "🦆", "🐔", "🐗"],
            "correct_answer": "🦆",
            "explanation": "Duck (🦆) is known to live in water, others are land animals."
        },
        {
            "question_id": "3",
            "question": "Which of these animals does not lay eggs?",
            "options": ["🐔", "🐧", "🐍", "🐵"],
            "correct_answer": "🐵",
            "explanation": "Monkeys (🐵) give birth to live young, they do not lay eggs."
        },
        {
            "question_id": "4",
            "question": "Which of these emojis 

In [101]:
import json
json.loads(output["output_text"])

{'questions': [{'question_id': '1',
   'question': 'Which of the following animals is not a pet?',
   'options': ['🐶', '🐱', '🐹', '🦁'],
   'correct_answer': '🦁',
   'explanation': 'A lion (🦁) is a wild animal and not typically kept as a pet.'},
  {'question_id': '2',
   'question': 'Which one of these animals is known to live in water?',
   'options': ['🐮', '🦆', '🐔', '🐗'],
   'correct_answer': '🦆',
   'explanation': 'Duck (🦆) is known to live in water, others are land animals.'},
  {'question_id': '3',
   'question': 'Which of these animals does not lay eggs?',
   'options': ['🐔', '🐧', '🐍', '🐵'],
   'correct_answer': '🐵',
   'explanation': 'Monkeys (🐵) give birth to live young, they do not lay eggs.'},
  {'question_id': '4',
   'question': 'Which of these emojis represents an animal that hibernates in winter?',
   'options': ['🐻', '🐴', '🐼', '🦒'],
   'correct_answer': '🐻',
   'explanation': 'Bears (🐻) are known to hibernate during the winter months.'},
  {'question_id': '5',
   'question

In [102]:
from typing import List
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

class Question(BaseModel):
    question_id: str = Field(..., description="ID of the question")
    question: str = Field(..., description="The question text")
    options: List[str] = Field(..., description="List of answer options")
    correct_answer: str = Field(..., description="The correct answer")
    explanation: str = Field(..., description="Explanation for the correct answer")

class Quiz(BaseModel):
    questions: List[Question] = Field(..., description="List of questions")

parser = PydanticOutputParser(pydantic_object=Quiz)

In [103]:
misformatted = """{
    "questions": [
        {
            "question_id": "1",
            "question": "Which one of the following is considered a pet?",
            "options": ["🦓", "🐘", "🐱", "🐊"],
            "correct_answer": "🐱",
            "explanation": "Out of these options, only a cat is typically kept as a pet."
        },
        {
            "question_id": "2",
            "question": "Which one of these animals usually lives in the jungle?",
            "options": ["🐱", "🐮", "🐵", "🐔"],
            "correct_answer": "🐵",
            "explanation": "Monkeys are known to inhabit jungles."
        },
        {
            "question_id": "3",
            "question": "Which animal is known to be a farm animal?",
            "options": ["🐘", "🦓", "🐮", "🐵"],
            "correct_answer": "🐮",
            "explanation": "Cows are commonly found on farms."
        },
        {
            "question_id": "4",
            "question": "Which animal is the largest?",
            "options": ["🐘", "🐱", "🐮", "🐵"],
            "correct_answer": "🐘",
            "explanation": "Elephants are the largest land animals."
        },
        {
            "question_id": "5",
            "question": "Which animal is not a mammal?",
            "options": ["🐘", "🐱", "🐔", "🐵"],
            "correct_answer": "🐔"
    ]
}"""

In [104]:
parser.parse(misformatted)

OutputParserException: Failed to parse Quiz from completion {
    "questions": [
        {
            "question_id": "1",
            "question": "Which one of the following is considered a pet?",
            "options": ["🦓", "🐘", "🐱", "🐊"],
            "correct_answer": "🐱",
            "explanation": "Out of these options, only a cat is typically kept as a pet."
        },
        {
            "question_id": "2",
            "question": "Which one of these animals usually lives in the jungle?",
            "options": ["🐱", "🐮", "🐵", "🐔"],
            "correct_answer": "🐵",
            "explanation": "Monkeys are known to inhabit jungles."
        },
        {
            "question_id": "3",
            "question": "Which animal is known to be a farm animal?",
            "options": ["🐘", "🦓", "🐮", "🐵"],
            "correct_answer": "🐮",
            "explanation": "Cows are commonly found on farms."
        },
        {
            "question_id": "4",
            "question": "Which

In [105]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
quiz = new_parser.parse(misformatted)
quiz

Quiz(questions=[Question(question_id='1', question='Which one of the following is considered a pet?', options=['🦓', '🐘', '🐱', '🐊'], correct_answer='🐱', explanation='Out of these options, only a cat is typically kept as a pet.'), Question(question_id='2', question='Which one of these animals usually lives in the jungle?', options=['🐱', '🐮', '🐵', '🐔'], correct_answer='🐵', explanation='Monkeys are known to inhabit jungles.'), Question(question_id='3', question='Which animal is known to be a farm animal?', options=['🐘', '🦓', '🐮', '🐵'], correct_answer='🐮', explanation='Cows are commonly found on farms.'), Question(question_id='4', question='Which animal is the largest?', options=['🐘', '🐱', '🐮', '🐵'], correct_answer='🐘', explanation='Elephants are the largest land animals.'), Question(question_id='5', question='Which animal is not a mammal?', options=['🐘', '🐱', '🐔', '🐵'], correct_answer='🐔', explanation='Chickens are birds, not mammals.')])

In [106]:
[{
    "question_id": j.question_id,
    "question": j.question,
    "options": j.options,
    "correct_answer": j.correct_answer,
    "explanation": j.explanation
} for j in quiz.questions]

[{'question_id': '1',
  'question': 'Which one of the following is considered a pet?',
  'options': ['🦓', '🐘', '🐱', '🐊'],
  'correct_answer': '🐱',
  'explanation': 'Out of these options, only a cat is typically kept as a pet.'},
 {'question_id': '2',
  'question': 'Which one of these animals usually lives in the jungle?',
  'options': ['🐱', '🐮', '🐵', '🐔'],
  'correct_answer': '🐵',
  'explanation': 'Monkeys are known to inhabit jungles.'},
 {'question_id': '3',
  'question': 'Which animal is known to be a farm animal?',
  'options': ['🐘', '🦓', '🐮', '🐵'],
  'correct_answer': '🐮',
  'explanation': 'Cows are commonly found on farms.'},
 {'question_id': '4',
  'question': 'Which animal is the largest?',
  'options': ['🐘', '🐱', '🐮', '🐵'],
  'correct_answer': '🐘',
  'explanation': 'Elephants are the largest land animals.'},
 {'question_id': '5',
  'question': 'Which animal is not a mammal?',
  'options': ['🐘', '🐱', '🐔', '🐵'],
  'correct_answer': '🐔',
  'explanation': 'Chickens are birds, not 